# Get Latitude and Longitude values from NASA satalite images

In [2]:
import h5py
import numpy as np

filename = '/Users/dazedinthecity/Desktop/Data_sci_proj/nightlights/test_in_tut_0402_monthly/VNP46A3.A2014182.h19v04.002.2025090185911.h5'

with h5py.File(filename, 'r') as f:
    # Extract tile info from filename
    h_tile = 19  # from filename h19v04
    v_tile = 4
    
    # Get the data to determine dimensions
    # VNP46A3 uses 'HDFEOS/GRIDS/VNP_Grid_DNB' structure
    data_key = list(f['HDFEOS/GRIDS'].keys())[0]  # Usually 'VNP_Grid_DNB'
    data = f[f'HDFEOS/GRIDS/{data_key}/Data Fields']
    
    # Get any dataset to determine shape
    dataset_name = list(data.keys())[0]
    shape = data[dataset_name].shape
    nrows, ncols = shape
    
    print(f"Data shape: {nrows} x {ncols}")
    
    # MODIS/VIIRS sinusoidal grid parameters
    tile_size = 1111950.5197665554  # meters per tile
    pixel_size = tile_size / nrows  # assuming square pixels
    R = 6371007.181  # Earth radius in meters
    
    # Create pixel coordinate arrays
    x = np.arange(ncols)
    y = np.arange(nrows)
    xx, yy = np.meshgrid(x, y)
    
    # Convert pixel coordinates to meters in sinusoidal projection
    # Origin is at upper-left of the global grid
    x_meters = (h_tile * tile_size) + (xx * pixel_size) - (18 * tile_size)
    y_meters = (9 * tile_size) - (v_tile * tile_size) - (yy * pixel_size)
    
    # Convert sinusoidal projection to lat/lon (WGS84)
    lat = y_meters / R * (180 / np.pi)
    
    # Longitude depends on latitude in sinusoidal projection
    lon = x_meters / (R * np.cos(lat * np.pi / 180)) * (180 / np.pi)
    
    print(f"\nLatitude range: {lat.min():.4f} to {lat.max():.4f}")
    print(f"Longitude range: {lon.min():.4f} to {lon.max():.4f}")
    print(f"\nLat/Lon arrays shape: {lat.shape}")
    
    # VNP46A3 specific datasets (monthly composite)
    print("\nAvailable datasets:")
    for ds_name in data.keys():
        print(f"  - {ds_name}")
    
    # Example: Load a data field and its corresponding coordinates
    # Common VNP46A3 datasets: 'NearNadir_Composite_Snow_Free'
    if 'NearNadir_Composite_Snow_Free' in data:
        ntl_data = data['NearNadir_Composite_Snow_Free'][:]
        print(f"\nLoaded NearNadir_Composite_Snow_Free: {ntl_data.shape}")
    
    # Now ntl_data[i, j] has coordinates lat[i, j], lon[i, j]
    
    # Optional: Save to file
    np.savez('coordinates_VNP46A3.npz', latitude=lat, longitude=lon)
    print("\nCoordinates saved to coordinates_VNP46A3.npz")

Data shape: 2400 x 2400

Latitude range: 40.0042 to 50.0000
Longitude range: 13.0549 to 31.1080

Lat/Lon arrays shape: (2400, 2400)

Available datasets:
  - AllAngle_Composite_Snow_Covered
  - AllAngle_Composite_Snow_Covered_Num
  - AllAngle_Composite_Snow_Covered_Quality
  - AllAngle_Composite_Snow_Covered_Std
  - AllAngle_Composite_Snow_Free
  - AllAngle_Composite_Snow_Free_Num
  - AllAngle_Composite_Snow_Free_Quality
  - AllAngle_Composite_Snow_Free_Std
  - DNB_Platform
  - Land_Water_Mask
  - NearNadir_Composite_Snow_Covered
  - NearNadir_Composite_Snow_Covered_Num
  - NearNadir_Composite_Snow_Covered_Quality
  - NearNadir_Composite_Snow_Covered_Std
  - NearNadir_Composite_Snow_Free
  - NearNadir_Composite_Snow_Free_Num
  - NearNadir_Composite_Snow_Free_Quality
  - NearNadir_Composite_Snow_Free_Std
  - OffNadir_Composite_Snow_Covered
  - OffNadir_Composite_Snow_Covered_Num
  - OffNadir_Composite_Snow_Covered_Quality
  - OffNadir_Composite_Snow_Covered_Std
  - OffNadir_Composite_Sno